- https://chat.qwen.ai/c/cd9c27bf-94c6-4d9f-8e1d-ffeb54d55ac8

In [1]:
!pip install pypinyin rapidfuzz

   ---------------------------------------- 0.0/837.0 kB ? eta -:--:--
   ------------------------- -------------- 524.3/837.0 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 837.0/837.0 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.6 MB 2.8 MB/s eta 0:00:01
   ------------------------- -------------- 1.0/1.6 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 2.9 MB/s eta 0:00:00


In [33]:
!cd 

C:\Users\p2p2l\projects\wgong\zistory\zinets\app\zadmin


In [22]:
from pypinyin import pinyin, Style
from rapidfuzz import distance

# Function to extract Pinyin and split into initial/final
def extract_pinyins(character, style=Style.NORMAL, heteronym=True):
    """
    Extract PinYin pronunciations (heteronyms) from character:
    support the following styles:
        - Style.NORMAL (default)
        - Style.TONE
        - Style.TONE2
        - Style.TONE3

    Returns a tuple of pinyins (拼音), initials (声母), finals (韵母)
    """
    
    pinyins = pinyin(character, style=style, heteronym=heteronym)[0]
    initials = pinyin(character, style=Style.INITIALS|style, heteronym=heteronym)[0]
    finals = pinyin(character, style=Style.FINALS|style, heteronym=heteronym)[0]
    return pinyins, initials, finals

def calculate_similarity(pron1, pron2):
    # Helper function to calculate similarity
    return 1 - distance.Levenshtein.normalized_distance(pron1, pron2)

def get_similarity(char, comp, threshold=0.4, style=Style.NORMAL, heteronym=True, debug=True):
    """
    Calculate pinyin similarity scores in 2 steps
    """
    
    max_raw_similarity, max_refined_similarity = -1, -1
    
    char_pinyins = extract_pinyins(char, style=style, heteronym=heteronym)
    phon_pinyins = extract_pinyins(comp, style=style, heteronym=heteronym)
    if debug:
        print(f"Character: {char_pinyins}")
        print(f"Phonetic Component: {phon_pinyins}")        

    # Step 1: Calculate raw similarity (full Pinyin)
    # for pron1 in char_pinyins[0]:
    #     for pron2 in phon_pinyins[0]:
    #         raw_similarity = calculate_similarity(pron1, pron2)
            
    #         # Update maximum raw similarity
    #         if raw_similarity > max_raw_similarity:
    #             max_raw_similarity = raw_similarity
    pron1 = char_pinyins[0][0]
    pron2 = phon_pinyins[0][0]
    max_raw_similarity = calculate_similarity(pron1, pron2)

    # Step 2: Calculate refined similarity (finals) if raw similarity is above threshold
    if max_raw_similarity >= threshold:
        for pron1 in char_pinyins[2]:
            for pron2 in phon_pinyins[2]:
                refined_similarity = calculate_similarity(pron1, pron2)
                                
                # Update maximum refined similarity
                if refined_similarity > max_refined_similarity:
                    max_refined_similarity = refined_similarity
    
    return f"{max_raw_similarity:.3f}", f"{max_refined_similarity:.3f}"


In [8]:
character = "行"

styles = [Style.NORMAL, Style.TONE, Style.TONE2, Style.TONE3]

for style in styles:
    x = pinyin(character, style=style, heteronym=True)[0]
    print(f"style={style}\n \tpinyins = {x}")

extract_pinyins(character, style=Style.NORMAL)

style=0
 	pinyins = ['xing', 'hang', 'heng']
style=1
 	pinyins = ['xíng', 'háng', 'héng', 'xìng', 'hàng']
style=2
 	pinyins = ['xi2ng', 'ha2ng', 'he2ng', 'xi4ng', 'ha4ng']
style=8
 	pinyins = ['xing2', 'hang2', 'heng2', 'xing4', 'hang4']


(['xing', 'hang', 'heng'], ['x', 'h'], ['ing', 'ang', 'eng'])

In [26]:
# Example
character, phonetic_component = "始", "厶"

x = get_similarity(character, phonetic_component, style=Style.NORMAL, heteronym=True)
print(x)

Character: (['shi'], ['sh'], ['i'])
Phonetic Component: (['si', 'mou'], ['s', 'm'], ['i', 'ou'])
('0.667', '1.000')


In [27]:
# Example
character, phonetic_component = "始", "台" 

x = get_similarity(character, phonetic_component, style=Style.NORMAL, heteronym=True)
print(x)

Character: (['shi'], ['sh'], ['i'])
Phonetic Component: (['tai', 'yi', 'si'], ['t', 's'], ['ai', 'i'])
('0.333', '-1.000')


In [28]:
# Example
character, phonetic_component = "始", "口"

x = get_similarity(character, phonetic_component, style=Style.NORMAL, heteronym=True)
print(x)

Character: (['shi'], ['sh'], ['i'])
Phonetic Component: (['kou'], ['k'], ['ou'])
('0.000', '-1.000')


In [29]:
# Example
character, phonetic_component = "行", "亍"

x = get_similarity(character, phonetic_component, style=Style.NORMAL, heteronym=True)
print(x)

Character: (['xing', 'hang', 'heng'], ['x', 'h'], ['ing', 'ang', 'eng'])
Phonetic Component: (['chu'], ['ch'], ['u'])
('0.000', '-1.000')


初: 始也。从刀从衣。裁衣之始也。

In [30]:
# Example
character, phonetic_component = "初", "衤"

x = get_similarity(character, phonetic_component, style=Style.NORMAL, heteronym=True)
print(x)

Character: (['chu'], ['ch'], ['u'])
Phonetic Component: (['yi'], [''], ['i'])
('0.000', '-1.000')


In [31]:
# Example
character, phonetic_component = "初", "衣"

x = get_similarity(character, phonetic_component, style=Style.NORMAL, heteronym=True)
print(x)

Character: (['chu'], ['ch'], ['u'])
Phonetic Component: (['yi'], [''], ['i'])
('0.000', '-1.000')


In [32]:
# Example
character, phonetic_component = "初", "刀"

x = get_similarity(character, phonetic_component, style=Style.NORMAL, heteronym=True)
print(x)

Character: (['chu'], ['ch'], ['u'])
Phonetic Component: (['dao', 'diao'], ['d'], ['ao', 'iao'])
('0.000', '-1.000')
